In [ ]:
# Need to convert from TTT to connect4

In [1]:
import pickle
import numpy as np
import re
from collections import Counter
from model import Board, RandomPlayer, MiniMaxPlayer, HumanPlayer, Player, search, minimax, draw_graph
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout

# Set up a player and blank board

In [20]:
# X goes first
player = Player('X')
board = Board(state=np.zeros((3,3), dtype=int))
board.draw(board.state)

-|-|-
-----
-|-|-
-----
-|-|-


In [21]:
# board = Board(state=np.array([[0,0,0],[0,1,0],[2,0,0]], dtype=int))
# board.draw(board.state)

# Minimax over full game space (depth=9)

In [22]:
%%time
G = search(board, player_marker='O', max_depth=9, player_moves_first=False)

CPU times: user 42.4 s, sys: 722 ms, total: 43.1 s
Wall time: 43.2 s


In [23]:
print(G.number_of_nodes())
leaf_nodes = [node for node in G.nodes() if G.in_degree(node)!=0 and G.out_degree(node)==0]
print(len(leaf_nodes))

549946
255168


In [24]:
# #Gs = G.subgraph(nx.dfs_tree(G, 0))
# Gs = G.subgraph(nx.dfs_tree(G, 0, depth_limit=3))
# Gs.number_of_nodes()
# draw_graph(Gs, fig_size=(5, 20), node_label='player', edge_label='move')

# Do minimax over all states

In [25]:
%%time
H = minimax(G, first_move=False)

CPU times: user 7.22 s, sys: 257 ms, total: 7.47 s
Wall time: 7.53 s


In [26]:
# #Gs = G.subgraph(nx.dfs_tree(G, 0))
# Hs = H.subgraph(nx.dfs_tree(H, 0, depth_limit=2))
# Hs.number_of_nodes()
# draw_graph(Hs, fig_size=(5, 20), node_label='player', edge_label='move')

In [27]:
# #Gs = G.subgraph(nx.dfs_tree(G, 0))
# Hs = H.subgraph(nx.dfs_tree(H, 800, depth_limit=4))
# Hs.number_of_nodes()
# draw_graph(Hs, fig_size=(5, 20), node_label='score', edge_label='move')

In [28]:
# Add best minimax move to each node
for n in H.nodes():
    scores = [(v, c['move'], H.nodes[v]['score']) for (u, v, c) in H.out_edges(n, data=True)]
    if scores:
        best_move = max(scores, key = lambda t: t[2])[1]
        H.nodes[n]['best_move'] = best_move

In [31]:
# Hash all best moves:
minimax_move = {}

def state_to_string(state) -> str:
    board_str = np.array_str(state.flatten())
    #value_to_marker = {0: '-', 1: 'X', 2: 'O'}
    replacements = [
        ('0', '-'),
        ('1', 'X'),
        ('2', 'O'),
        ('\[', ''),
        ('\]', ''),
        (' ', ''),
    ]
    for old, new in replacements:
        board_str = re.sub(old, new, board_str)

    return board_str

def best_move_to_int(best_move:tuple) -> int:
    return int(1+(3*best_move[0])+best_move[1])

for node in H.nodes(data=True):
    if node[1].get('best_move') and node[1].get('player') == 'max':
        #minimax_move[node[1]['state'].tostring()] = node[1]['best_move']
        minimax_move[state_to_string(node[1]['state'])] = best_move_to_int(node[1]['best_move'])

## Save minimax dictionary to disk

In [32]:
import json
with open('/Users/davidrossouw/Documents/my_projects/cloud-run/tic_tac_toe/ttt_minimax.json', 'w') as f:
    json.dump(minimax_move, f)

In [34]:
minimax_move['X--------']

5